In [16]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
import torch
import yaml
from dotenv import load_dotenv
import chromadb
import pandas as pd
import os

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device}.")

Using cuda.


In [3]:
splits = {'train': 'data/train-00000-of-00001-b1700331af6d3576.parquet', 'test': 'data/test-00000-of-00001-460abe60f17dbc1c.parquet'}
df = pd.read_parquet("hf://datasets/ShashiVish/cover-letter-dataset/" + splits["train"])

In [4]:
df

,Job Title,Preferred Qualifications,Hiring Company,Applicant Name,Past Working Experience,Current Working Experience,Skillsets,Qualifications,Cover Letter
0,Senior Java Developer,5+ years of experience in Java Development,Google,John Doe,Java Developer at XYZ for 3 years,Senior Java Developer at ABC for 2 years,"Java, Spring Boot, Hibernate, SQL",BSc in Computer Science,I am writing to express my interest in the Sen...
1,Data Scientist,5 years of experience in data modeling,XYZ Analytics Solutions,John Smith,Data Analyst at ABC Corporation,Data Scientist at XYZ Technologies,"Data modeling, data analysis, programming (Pyt...","Bachelor's degree in Computer Science, Master'...","Dear Hiring Manager,\n\nI am writing to expres..."
2,Data Scientist,Experience with Python and proficiency in at l...,XYZ Analytics,John Smith,Data Analyst at ABC Corporation,Data Scientist at XYZ Solutions,"Python, R, SQL, Machine Learning, Statistical ...","Bachelor's degree in Computer Science, Master'...","Dear Hiring Manager,\n\nI am writing to expres..."
3,Senior Data Scientist,Minimum of 3 years validated experience\nOutst...,XYZ Tech Solutions,John Smith,Data Analyst at ABC Company,Senior Data Scientist at DEF Corporation,"Python, machine learning libraries, deep learn...","Bachelor's degree in Computer Science, Master'...","Dear Hiring Manager,\n\nI am writing to expres..."
4,data scientist,35 years+ professional experience in data sci...,xyz tech solutions,john smith,data analyst at abc company,data scientist at xyz tech solutions,"Python, R, Machine Learning, NLP, Data Visuali...",35 years+ professional experience in data sci...,"Dear Hiring Manager,\n\nI am writing to expres..."
...,...,...,...,...,...,...,...,...,...
808,Senior Support Engineer,"5+ years of customer support, ITIL",IBM,David Davis,Support Engineer at XYZ for 2 years,Senior Support Engineer at ABC for 3 years,"Customer Support, ITIL, Troubleshooting, SQL, ...",Bachelor's in Information Technology,I am applying for the Senior Support Engineer ...
809,Network Engineer,Experience with VoIP technologies and network ...,Apple,Edward Hernandez,Network Engineer at Google (5 years),Senior Network Engineer at Amazon (3 years),"VoIP Technologies, Network Security, Cisco, Ju...",BSc in Network Engineering,I am applying for the Network Engineer positio...
810,Mobile App Developer,Master's degree in Computer Science or related...,XYZ Company,Jennifer Wilson,5 years of mobile app development experience,4 years of mobile app development experience,"iOS, Android, Swift, Java, Kotlin","Excellent communication skills, ability to wor...","Dear Hiring Manager, I am writing to express m..."
811,Data Scientist,"University degree in engineering, data, or re...",XYZ Corporation,John Smith,Data Analyst at ABC Company,Data Scientist at DEF Company,"Data analysis, machine learning, programming (...","Bachelor's degree in Computer Science, PMP Cer...","Dear Hiring Manager,\n\nI am writing to expres..."


In [9]:
df.describe()

,Job Title,Preferred Qualifications,Hiring Company,Applicant Name,Past Working Experience,Current Working Experience,Skillsets,Qualifications,Cover Letter
count,813,811,813,813,813,813,806,805,813
unique,56,540,159,263,345,379,482,215,760
top,Data Scientist,PhD in Computer Science,XYZ Analytics,John Smith,Data Analyst at ABC Company,Data Scientist at DEF Company,"Python, R, SQL, Machine Learning, Deep Learning",Bachelor's degree in Computer Science,I am writing to express my interest in the Sen...
freq,249,33,84,365,125,56,35,56,4


In [10]:
df.isna().sum()

Job Title                     0
Preferred Qualifications      2
Hiring Company                0
Applicant Name                0
Past Working Experience       0
Current Working Experience    0
Skillsets                     7
Qualifications                8
Cover Letter                  0
dtype: int64

In [12]:
df.dropna(inplace=True)

In [13]:
df

,Job Title,Preferred Qualifications,Hiring Company,Applicant Name,Past Working Experience,Current Working Experience,Skillsets,Qualifications,Cover Letter
0,Senior Java Developer,5+ years of experience in Java Development,Google,John Doe,Java Developer at XYZ for 3 years,Senior Java Developer at ABC for 2 years,"Java, Spring Boot, Hibernate, SQL",BSc in Computer Science,I am writing to express my interest in the Sen...
1,Data Scientist,5 years of experience in data modeling,XYZ Analytics Solutions,John Smith,Data Analyst at ABC Corporation,Data Scientist at XYZ Technologies,"Data modeling, data analysis, programming (Pyt...","Bachelor's degree in Computer Science, Master'...","Dear Hiring Manager,\n\nI am writing to expres..."
2,Data Scientist,Experience with Python and proficiency in at l...,XYZ Analytics,John Smith,Data Analyst at ABC Corporation,Data Scientist at XYZ Solutions,"Python, R, SQL, Machine Learning, Statistical ...","Bachelor's degree in Computer Science, Master'...","Dear Hiring Manager,\n\nI am writing to expres..."
3,Senior Data Scientist,Minimum of 3 years validated experience\nOutst...,XYZ Tech Solutions,John Smith,Data Analyst at ABC Company,Senior Data Scientist at DEF Corporation,"Python, machine learning libraries, deep learn...","Bachelor's degree in Computer Science, Master'...","Dear Hiring Manager,\n\nI am writing to expres..."
4,data scientist,35 years+ professional experience in data sci...,xyz tech solutions,john smith,data analyst at abc company,data scientist at xyz tech solutions,"Python, R, Machine Learning, NLP, Data Visuali...",35 years+ professional experience in data sci...,"Dear Hiring Manager,\n\nI am writing to expres..."
...,...,...,...,...,...,...,...,...,...
808,Senior Support Engineer,"5+ years of customer support, ITIL",IBM,David Davis,Support Engineer at XYZ for 2 years,Senior Support Engineer at ABC for 3 years,"Customer Support, ITIL, Troubleshooting, SQL, ...",Bachelor's in Information Technology,I am applying for the Senior Support Engineer ...
809,Network Engineer,Experience with VoIP technologies and network ...,Apple,Edward Hernandez,Network Engineer at Google (5 years),Senior Network Engineer at Amazon (3 years),"VoIP Technologies, Network Security, Cisco, Ju...",BSc in Network Engineering,I am applying for the Network Engineer positio...
810,Mobile App Developer,Master's degree in Computer Science or related...,XYZ Company,Jennifer Wilson,5 years of mobile app development experience,4 years of mobile app development experience,"iOS, Android, Swift, Java, Kotlin","Excellent communication skills, ability to wor...","Dear Hiring Manager, I am writing to express m..."
811,Data Scientist,"University degree in engineering, data, or re...",XYZ Corporation,John Smith,Data Analyst at ABC Company,Data Scientist at DEF Company,"Data analysis, machine learning, programming (...","Bachelor's degree in Computer Science, PMP Cer...","Dear Hiring Manager,\n\nI am writing to expres..."


In [14]:
import json

dataset_json = df.to_dict(orient="records")

In [19]:
try:
    if not os.path.exists("./dataset"):
        os.makedirs("./dataset")
    with open("dataset/cover_letter_dataset.json", "w") as f:
        json.dump(dataset_json, f, indent=4)
    print("Dataset saved as JSON.")
    
except FileNotFoundError as e:
    print(f"Error: {e}")

Dataset saved as JSON.


In [ ]:
model_google = "google/flan-t5-large"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_google)
model = AutoModelForSeq2SeqLM.from_pretrained(model_google)
model.to(device)

In [ ]:
chroma_client = chromadb.PersistentClient(path="./cover_letter_db")

In [ ]:
chroma_client.create_collection(name="cover_letters")

In [ ]:
collection = chroma_client.get_collection(name="cover_letters")

In [ ]:
collection.count()

In [ ]:
results = collection.query(
    collection_name = "cover_letters",
    query_texts = ["Data Scientist at Microsoft"],
    n_result = 1
)

In [ ]:
pipeline = pipeline('text2text-generation', model=model_google)

In [ ]:
prompt = '''
Write a professional cover letter for a Data Scientist position at Microsoft.
The applicant has 5 years of experience in Python, AI, and Data Science.
Make it formal, well-structured, and persuasive.
'''

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, min_length = 400, max_length=1024, temperature=0.7, top_p = 0.95, do_sample = True)

In [ ]:
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)